In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [11]:
class LinkedInScraper:
    def __init__(self, email, password):
        self.email = email
        self.password = password
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    def login(self):
        self.driver.get('https://www.linkedin.com/login')
        time.sleep(2)

        email_elem = self.driver.find_element(By.ID, "username")
        email_elem.send_keys(self.email)

        password_elem = self.driver.find_element(By.ID, "password")
        password_elem.send_keys(self.password)
        password_elem.send_keys(Keys.RETURN)
        time.sleep(2)
        input("Please complete the captcha and login manually, then press Enter to continue...")


    def fetch_jobs(self, entries, output_file):
        with open(output_file, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["company_name", 'company_link', "job_title_location_and_work_type"])  # Header

            for entry in entries:
                url = entry['company_domain']
                if url:
                    self.driver.get(url)
                    time.sleep(3)  # Allow time for all scripts to load and data to display

                    # Handle pagination
                    try:
                        link = self.driver.find_element(By.CSS_SELECTOR, "a.artdeco-button.artdeco-button--primary.artdeco-button--3.flex-shrink-zero")
                        link.click()
                        time.sleep(4)  # Wait for the new tab to open

                        original_window = self.driver.current_window_handle
                        new_window = [window for window in self.driver.window_handles if window != original_window][0]
                        self.driver.switch_to.window(new_window)
                        time.sleep(3)  # Allow some time for the new tab to load content
                        
                        current_page = 1
                        anchor_tag_case = 1

                        while True:
                            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                            ul_tag = soup.find('ul', class_='scaffold-layout__list-container')
                            if ul_tag:
                                li_tags = ul_tag.find_all('li')
                                for li in li_tags:
                                    div_tag = li.find('div', class_='flex-grow-1 artdeco-entity-lockup__content ember-view')
                                    if div_tag:
                                        job_title = div_tag.find('strong')
                                        company_name = div_tag.find('span', class_='job-card-container__primary-description')
                                        country_and_work_type = div_tag.find('li', class_='job-card-container__metadata-item')
                                        company_url = url
                                        
                                        job_title = (job_title.text).strip().replace('\n', '') if job_title else "N/A"
                                        country_and_work_type = (country_and_work_type.text).strip().replace('\n', '') if country_and_work_type else "N/A"
                                        job_title_location_and_work_type = job_title + " - " + country_and_work_type

                                        writer.writerow([
                                            (company_name.text).strip().replace('\n', '') if company_name else "N/A",
                                            company_url,
                                            job_title_location_and_work_type
                                        ])  
                            else:
                                print("UL tag not found for company URL: ", url)
                                break

                            # Check for next page
                            current_page += 1
                            next_button_selector = f"button[aria-label='Page {current_page}']"
                            next_buttons = self.driver.find_elements(By.CSS_SELECTOR, next_button_selector)

                            if next_buttons:
                                # print(next_buttons[0], 'next button is here')
                                next_buttons[0].click()
                                time.sleep(5)  # Wait for the next page to load
                                print('next page')
                            else:
                                print("No more pages to load.")
                                break

                    except NoSuchElementException:
                        writer.writerow([
                                        
                                        entry['company_name'],
                                        entry['company_domain'],
                                        ''
                                    ]),
                        print("The specific anchor tag is not found")
                        anchor_tag_case = 0
                        
                    except Exception as e:
                        print(f"An error occurred: {e}")
                        
                    finally:
                        print('done loading')
                        if anchor_tag_case == 1:
                            self.driver.close()  # Close the new window
                            if original_window in self.driver.window_handles:
                                self.driver.switch_to.window(original_window)
                            else:
                                print("Original window has been closed.")
                    anchor_tag_case = 1        
                    time.sleep(1)  # Short wait before processing next entry
                            
            self.driver.quit()

def read_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        entries = [row for row in reader]
    return entries

# Usage
username = "akber.anwer1234@gmail.com" # linkedin username
password = "Makber@8472" # linkedin password
input_file = 'test_input_file.csv' # column name -> company_name, company_domain
output_file = 'test_output_file.csv' # no need for columns, it will be automatically created

scraper = LinkedInScraper(username, password)
scraper.login()

entries = read_csv(input_file)

scraper.fetch_jobs(entries, output_file)

The specific anchor tag is not found
done loading
No more pages to load.
done loading
next page
No more pages to load.
done loading
No more pages to load.
done loading
next page
next page
No more pages to load.
done loading
No more pages to load.
done loading
next page
next page
next page
next page
next page
No more pages to load.
done loading
No more pages to load.
done loading
No more pages to load.
done loading
next page
next page
next page
next page
next page
next page
No more pages to load.
done loading
The specific anchor tag is not found
done loading


In [12]:
input_file = 'test_output_file.csv'
df = pd.read_csv(input_file)
df.head()

,company_name,company_link,job_title_location_and_work_type
0,TestCase,https://www.linkedin.com/company/hawklogix-pak...,NaN
1,Abrigo,https://www.linkedin.com/company/weareabrigo/j...,"Cloud Security Engineer I - Raleigh, NC (On-site)"
2,Abrigo,https://www.linkedin.com/company/weareabrigo/j...,Senior Data Scientist - United States (Remote)
3,Abrigo,https://www.linkedin.com/company/weareabrigo/j...,Principal Data Engineer - United States (Remote)
4,Abrigo,https://www.linkedin.com/company/weareabrigo/j...,Senior Cloud Operations Engineer - United Stat...


In [13]:
# Group by 'company_name' and 'company_link', aggregate 'job_title' and 'country_and_work_type'
# Convert NaN values to empty strings
df['job_title_location_and_work_type'] = df['job_title_location_and_work_type'].fillna('')
# df['country_and_work_type'] = df['country_and_work_type'].fillna('')
df['company_link'] = df['company_link'].apply(lambda x: x.replace('jobs/', ''))

grouped = df.groupby(['company_name', 'company_link']).agg({
    'job_title_location_and_work_type': lambda x: '\n'.join(x),
    # 'country_and_work_type': lambda x: '\n'.join(x)
}).reset_index()

# Save the result to CSV and Excel files
grouped.to_csv('grouped_data.csv', index=False)
grouped.to_excel('grouped_data.xlsx', index=False)


In [65]:
import pandas as pd

# List of file names to merge
file_names = ['book_1_to_128_jd.csv', 'book_128_to_144_jd.csv', 'book_144_to_all_jd.csv']

# Read each file into a DataFrame
dfs = [pd.read_csv(file) for file in file_names]

# Merge the DataFrames
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a CSV file
merged_df.to_csv('merged_file.csv', index=False)


In [66]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('merged_file.csv')

# Function to extract and remove nature of job from the string
def extract_nature_of_job(job_string):
    job_string = job_string.strip()
    nature_of_job = ""
    if job_string.endswith("(On-site)"):
        nature_of_job = "On-site"
        job_string = job_string[:-len("(On-site)")].strip()
    elif job_string.endswith("(Remote)"):
        nature_of_job = "Remote"
        job_string = job_string[:-len("(Remote)")].strip()
    elif job_string.endswith("(Hybrid)"):
        nature_of_job = "Hybrid"
        job_string = job_string[:-len("(Hybrid)")].strip()
    return job_string, nature_of_job

# Process each entry in the specified column
new_column_values = []
for entry in df['Country_And_Work_Type']:
    new_entry, nature_of_job = extract_nature_of_job(entry)
    new_column_values.append(nature_of_job)
    # Update the original entry in the DataFrame
    df['Country_And_Work_Type'] = df['Country_And_Work_Type'].apply(lambda x: x.replace('(On-site)', ''))
    df['Country_And_Work_Type'] = df['Country_And_Work_Type'].apply(lambda x: x.replace('(Remote)', ''))
    df['Country_And_Work_Type'] = df['Country_And_Work_Type'].apply(lambda x: x.replace('(Hybrid)', ''))

# Add the new column to the DataFrame
df['nature_of_job'] = new_column_values

# Add the new column to the DataFrame
df['linked_link'] = ''

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False)

In [67]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('updated_file.csv')

# Fill NaN values with empty strings
df.fillna('', inplace=True)


# Rename columns
df.rename(columns={'Company': 'company_name', 
                   'Job_Title': 'job_title', 
                   'Country_And_Work_Type': 'country',
                   'nature_of_job': 'job_type'}, inplace=True)

C:\Users\akber\AppData\Local\Temp\ipykernel_63884\4018384694.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [69]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('updated_file.csv')
df2 = pd.read_csv('book_1_to_all.csv')

# Merge the DataFrames based on the "name" column
merged_df = pd.merge(df1, df2, on='company_name', how='inner')

# Update the "linkedin_link" column with the "domain" values where the names match
merged_df.loc[merged_df['company_name'].notnull(), 'company_link'] = merged_df['Domain']

# Drop the "domain" column as it is no longer needed
merged_df.drop('Domain', axis=1, inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('updated_file_with_company_link.csv', index=False)


ValueError: You are trying to merge on object and float64 columns for key 'company_name'. If you wish to proceed you should use pd.concat

In [71]:
# Read the CSV files into DataFrames
df = pd.read_csv('updated_file_with_company_link.csv')

df['company_link'] = df['company_link'].apply(lambda x: x.replace('jobs/', ''))

df.to_csv('updated_file_with_company_link_again.csv', index=False)
df.to_excel('updated_file_with_company_link_again.xlsx', index=False)
